In [1]:
import time
from tools.Iforest import Iforest

param = [5,6,7,8,9,10]

for feature in ['all','real','reactive','harmonic']:
    AUC_Scores = []
    times = []

    print('----------')
    print('Features used: ' + feature)
    
    for max_samples in param:
        mdl = Iforest(feature=feature, suppress_progress=True)
        mdl.max_samples = max_samples
        
        start = time.time()
        anomaly_score, y = mdl.cluster_all()
        end = time.time()
        
        sc = round(mdl.auc_score(anomaly_score, y), 3)
        AUC_Scores.append(sc)

        avrg_run = round(( (end - start)/1000 ), 2)
        times.append( avrg_run )

    print(max(AUC_Scores))
    print(AUC_Scores)
    print(times)
    print(param)

    print('----------')

----------
Features used: all


KeyboardInterrupt: 